In [1]:
import requests
import base64

CLIENT_ID = 'e923ef017da94c6684a623427e17d377'
CLIENT_SECRET = '11283a104ab54e2884ec0ac75fa41a34'

client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
!pip install spotipy

In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlists_data(playlist_ids, access_token):
    sp = spotipy.Spotify(auth=access_token)

    all_music_data = []

    for playlist_id in playlist_ids:
        playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

        for track_info in playlist_tracks['items']:
            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']

            audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

            try:
                album_info = sp.album(album_id) if album_id != 'Not available' else None
                release_date = album_info['release_date'] if album_info else None
            except:
                release_date = None

            try:
                track_info = sp.track(track_id) if track_id != 'Not available' else None
                popularity = track_info['popularity'] if track_info else None
            except:
                popularity = None

            track_data = {
                'Track Name': track_name,
                'Artists': artists,
                'Album Name': album_name,
                'Album ID': album_id,
                'Track ID': track_id,
                'Popularity': popularity,
                'Release Date': release_date,
                'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
                'Explicit': track_info.get('explicit', None),
                'External URLs': track_info.get('external_urls', {}).get('spotify', None),
                'Danceability': audio_features['danceability'] if audio_features else None,
                'Energy': audio_features['energy'] if audio_features else None,
                'Key': audio_features['key'] if audio_features else None,
                'Loudness': audio_features['loudness'] if audio_features else None,
                'Mode': audio_features['mode'] if audio_features else None,
                'Speechiness': audio_features['speechiness'] if audio_features else None,
                'Acousticness': audio_features['acousticness'] if audio_features else None,
                'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
                'Liveness': audio_features['liveness'] if audio_features else None,
                'Valence': audio_features['valence'] if audio_features else None,
                'Tempo': audio_features['tempo'] if audio_features else None,
            }

            all_music_data.append(track_data)

    df = pd.DataFrame(all_music_data)

    return df

playlist_ids = ['37i9dQZF1DX76Wlfdnj7AP', '0W3jWYa1sQm5U8Ds4E8hfK', '37i9dQZF1DXdpQPPZq3F7n']

music_df = get_trending_playlists_data(playlist_ids, access_token)

print(music_df)

                                  Track Name  \
0                                Not Like Us   
1                                    Houdini   
2                         I Don't Wanna Wait   
3                 BAND4BAND (feat. Lil Baby)   
4                                      Pedro   
..                                       ...   
227                                 Tere Bin   
228                               Tu Chahiye   
229             Aabaad Barbaad (From "Ludo")   
230                             Jag Ghoomeya   
231  Tumhe Kitna Pyaar Karte (From "Bawaal")   

                                               Artists  \
0                                       Kendrick Lamar   
1                                               Eminem   
2                            David Guetta, OneRepublic   
3                                Central Cee, Lil Baby   
4              Jaxomy, Agatino Romero, Raffaella Carrà   
..                                                 ...   
227   Rahat Fateh

In [5]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [6]:
print(music_df.columns)

Index(['Track Name', 'Artists', 'Album Name', 'Album ID', 'Track ID',
       'Popularity', 'Release Date', 'Duration (ms)', 'Explicit',
       'External URLs', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo'],
      dtype='object')


In [7]:
music_df.to_csv("Spotify_data.csv")

In [8]:
print(music_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        232 non-null    object 
 1   Artists           232 non-null    object 
 2   Album Name        232 non-null    object 
 3   Album ID          232 non-null    object 
 4   Track ID          232 non-null    object 
 5   Popularity        232 non-null    int64  
 6   Release Date      232 non-null    object 
 7   Duration (ms)     232 non-null    int64  
 8   Explicit          232 non-null    bool   
 9   External URLs     232 non-null    object 
 10  Danceability      232 non-null    float64
 11  Energy            232 non-null    float64
 12  Key               232 non-null    int64  
 13  Loudness          232 non-null    float64
 14  Mode              232 non-null    int64  
 15  Speechiness       232 non-null    float64
 16  Acousticness      232 non-null    float64
 1

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
data = music_df

In [11]:
def calculate_weighted_popularity(release_date):
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    time_span = datetime.now() - release_date

    weight = 1 / (time_span.days + 1)
    return weight

In [12]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [13]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [14]:
import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'I'm Good (Blue)':
                     Track Name                                     Artists  \
3  It's Not Right But It's Okay                            Mr. Belt & Wezol   
0                   In My Bones             Lost Frequencies, David Kushner   
1                         REACT  Switch Disco, Ella Henderson, Robert Miles   
4         Rainfall (Praise You)                                   Tom Santa   
2                 Where You Are                          John Summit, HAYLA   

                     Album Name Release Date  Popularity  
3  It's Not Right But It's Okay   2024-02-23        83.0  
0                   In My Bones   2024-04-19        74.0  
1                         REACT   2023-01-13        74.0  
4         Rainfall (Praise You)   2022-02-18        74.0  
2                 Where You Are   2023-03-03        71.0  


In [16]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Recommended songs for '{input_song_name}':")
print(recommendations)

Recommended songs for 'I'm Good (Blue)':
                     Track Name                                     Artists  \
3  It's Not Right But It's Okay                            Mr. Belt & Wezol   
0                   In My Bones             Lost Frequencies, David Kushner   
1                         REACT  Switch Disco, Ella Henderson, Robert Miles   
4         Rainfall (Praise You)                                   Tom Santa   
2                 Where You Are                          John Summit, HAYLA   

                     Album Name Release Date  Popularity  
3  It's Not Right But It's Okay   2024-02-23        83.0  
0                   In My Bones   2024-04-19        74.0  
1                         REACT   2023-01-13        74.0  
4         Rainfall (Praise You)   2022-02-18        74.0  
2                 Where You Are   2023-03-03        71.0  
